In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import sqlite3

In [33]:
#checking the tables in the sqlite_db

db = sqlite3.connect('hop_teaming.sqlite')
db.execute("""SELECT 
    name
FROM 
    sqlite_schema
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';""").fetchall()

[('hop',), ('npi_data',), ('taxonomy',), ('zipcode',)]

In [13]:
nppes = pd.read_excel('NPPES Deactivated NPI Report 20230213.xlsx')

C:\Users\tjsim\Anaconda\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [72]:
npi = pd.read_csv('npidata_pfile_20050523-20230212.csv', chunksize = 10000)

In [73]:
npi = next(npi)

C:\Users\tjsim\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3309: DtypeWarning: Columns (17,24,64,68,71,72,73,74,75,76,77,78,79,80,81,82,83,85,86,87,89,90,147,151,155,157,159,161,163,165,167,169,170,171,174,175,178,314) have mixed types.Specify dtype option on import or set low_memory=False.
  if await self.run_code(code, result, async_=asy):


In [76]:
npi.loc[npi['Entity Type Code'] == 1]

,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,...,Healthcare Provider Taxonomy Group_7,Healthcare Provider Taxonomy Group_8,Healthcare Provider Taxonomy Group_9,Healthcare Provider Taxonomy Group_10,Healthcare Provider Taxonomy Group_11,Healthcare Provider Taxonomy Group_12,Healthcare Provider Taxonomy Group_13,Healthcare Provider Taxonomy Group_14,Healthcare Provider Taxonomy Group_15,Certification Date
555,1508869488,1.0,NaN,NaN,NaN,HETHERINGTON,JOHN,J,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
556,1861495749,1.0,NaN,NaN,NaN,NASH,FRANK,M,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
557,1164425039,1.0,NaN,NaN,NaN,LEMCHEN,LAWRENCE,ARNOLD,MR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
558,1982607859,1.0,NaN,NaN,NaN,GIST,RAYMOND,F,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
559,1528061496,1.0,NaN,NaN,NaN,BRODY,HOWARD,NaN,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1558360792,1.0,NaN,NaN,NaN,RODGERS,SUSAN,V,MRS.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,1467451609,1.0,NaN,NaN,NaN,HILTON,BRET,D,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,1346249596,1.0,NaN,NaN,NaN,STANLEY,GARY,E,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9998,1982603130,1.0,NaN,NaN,NaN,SLETTEN,CHRISTINA,A.,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
nucc = pd.read_csv('nucc_taxonomy_230.csv', encoding = 'iso-8859-1')

In [19]:
nucc

,Code,Grouping,Classification,Specialization,Definition,Notes,Display Name,Section
0,193200000X,Group,Multi-Specialty,NaN,A business group of one or more individual pra...,[7/1/2003: new],Multi-Specialty Group,Individual
1,193400000X,Group,Single Specialty,NaN,A business group of one or more individual pra...,[7/1/2003: new],Single Specialty Group,Individual
2,207K00000X,Allopathic & Osteopathic Physicians,Allergy & Immunology,NaN,An allergist-immunologist is trained in evalua...,"Source: American Board of Medical Specialties,...",Allergy & Immunology Physician,Individual
3,207KA0200X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Allergy,"A physician who specializes in the diagnosis, ...",Source: National Uniform Claim Committee,Allergy Physician,Individual
4,207KI0005X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Clinical & Laboratory Immunology,An allergy and immunology physician who specia...,"Source: National Uniform Claim Committee, 2022...",Clinical & Laboratory Immunology (Allergy & Im...,Individual
...,...,...,...,...,...,...,...,...
868,343800000X,Transportation Services,Secured Medical Transport (VAN),NaN,A public or privately owned transportation ser...,NaN,Secured Medical Transport (VAN),Non-Individual
869,344600000X,Transportation Services,Taxi,NaN,A land commercial vehicle used for the transpo...,NaN,Taxi,Non-Individual
870,347D00000X,Transportation Services,Train,NaN,An organization or business licensed to provid...,NaN,Train,Non-Individual
871,347E00000X,Transportation Services,Transportation Broker,NaN,An organization that provides transportation f...,Source: Section 6083 of the Deficit Reduction ...,Transportation Broker,Non-Individual


In [28]:
hop = pd.read_csv('DocGraph_Hop_Teaming_2018.csv', chunksize = 10000)

In [29]:
next(hop)

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1508062167,1730166109,350,370,53.922,72.612
1,1508065640,1730166109,25,25,49.800,55.006
2,1508052093,1730166109,16,16,109.500,70.593
3,1508172545,1730166109,14,14,103.357,75.483
4,1508285131,1730166109,20,21,89.952,89.880
...,...,...,...,...,...,...
9995,1508026980,1730477589,36,46,64.239,77.845
9996,1497997050,1730477589,38,43,40.395,60.289
9997,1497977268,1730477761,14,14,37.857,72.533
9998,1508024217,1730477811,12,12,59.000,46.043


In [2]:
db = sqlite3.connect('hop_teaming.sqlite') #name sqlite then add dbs

for chunk in tqdm(pd.read_csv('DocGraph_Hop_Teaming_2018.csv', chunksize = 10000)):
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]
    chunk[(chunk['transaction_count'] >= 50) & (chunk['average_day_wait']<50)] # Clean up the column names
    chunk.to_sql('hop', db, if_exists = 'append', index = False)             # Append the chunk to a calls table

0it [00:00, ?it/s]

In [3]:
db.close

<function Connection.close>

In [4]:
npi_columns = ['NPI', 
    'Entity Type Code',
    'Provider Organization Name (Legal Business Name)',
    'Provider Last Name (Legal Name)',
    'Provider First Name',
    'Provider Middle Name',
    'Provider Name Prefix Text',
    'Provider Name Suffix Text',
    'Provider Credential Text',
    'Provider First Line Business Practice Location Address',
    'Provider Second Line Business Practice Location Address',
    'Provider Business Practice Location Address City Name',
    'Provider Business Practice Location Address State Name',
    'Provider Business Practice Location Address Postal Code',
    'Healthcare Provider Taxonomy Code_1',
    'Healthcare Provider Taxonomy Code_2',
    'Healthcare Provider Taxonomy Code_3',
    'Healthcare Provider Taxonomy Code_4',
    'Healthcare Provider Taxonomy Code_5',
    'Healthcare Provider Taxonomy Code_6',
    'Healthcare Provider Taxonomy Code_7',
    'Healthcare Provider Taxonomy Code_8',
    'Healthcare Provider Taxonomy Code_9',
    'Healthcare Provider Taxonomy Code_10',
    'Healthcare Provider Taxonomy Code_11',
    'Healthcare Provider Taxonomy Code_12',
    'Healthcare Provider Taxonomy Code_13',
    'Healthcare Provider Taxonomy Code_14',
    'Healthcare Provider Taxonomy Code_15',
    'Healthcare Provider Primary Taxonomy Switch_1',
    'Healthcare Provider Primary Taxonomy Switch_2',
    'Healthcare Provider Primary Taxonomy Switch_3',
    'Healthcare Provider Primary Taxonomy Switch_4',
    'Healthcare Provider Primary Taxonomy Switch_5',
    'Healthcare Provider Primary Taxonomy Switch_6',
    'Healthcare Provider Primary Taxonomy Switch_7',
    'Healthcare Provider Primary Taxonomy Switch_8',
    'Healthcare Provider Primary Taxonomy Switch_9',
    'Healthcare Provider Primary Taxonomy Switch_10',
    'Healthcare Provider Primary Taxonomy Switch_11',
    'Healthcare Provider Primary Taxonomy Switch_12',
    'Healthcare Provider Primary Taxonomy Switch_13',
    'Healthcare Provider Primary Taxonomy Switch_14',
    'Healthcare Provider Primary Taxonomy Switch_15']

In [5]:
npi_keeper_columns = ['NPI', 
    'Entity Type Code',
    'Provider Organization Name (Legal Business Name)',
    'Provider Last Name (Legal Name)',
    'Provider First Name',
    'Provider Middle Name',
    'Provider Name Prefix Text',
    'Provider Name Suffix Text',
    'Provider Credential Text',
    'Provider First Line Business Practice Location Address',
    'Provider Second Line Business Practice Location Address',
    'Provider Business Practice Location Address City Name',
    'Provider Business Practice Location Address State Name',
    'Provider Business Practice Location Address Postal Code',
    'taxonomy_code']

In [9]:
db = sqlite3.connect('hop_teaming.sqlite')

for chunk in tqdm(pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = npi_columns, chunksize = 10000)): #Reading data in by chuncks
    list_df = [] 

    for i in range (1,16): #Searching for the taxonomy code and creating its own column
        yes = npi[
                    npi[f'Healthcare Provider Primary Taxonomy Switch_{i}'] == 'Y'
                ][['NPI', f'Healthcare Provider Taxonomy Code_{i}']
                 ].rename(columns={f'Healthcare Provider Taxonomy Code_{i}':'taxonomy_code'})

        list_df.append(yes) 
    
    final = pd.merge(left = npi, right = pd.concat(list_df), how = 'left', on = 'NPI')[npi_keeper_columns] #Merging the taxonomy codes back to npi data 
    
    final.columns = [x.lower().replace(' ', '_') for x in final.columns] #cleaning column names
    final.to_sql('npi_data', db, if_exists = 'append', index = False) #building database



0it [00:00, ?it/s]

C:\Users\tjsim\AppData\Local\Temp\ipykernel_3808\2338586223.py:3: DtypeWarning: Columns (71,74,75,78,79,82,83,86,87,90) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = npi_columns, chunksize = 10000)): #Reading data in by chuncks
C:\Users\tjsim\AppData\Local\Temp\ipykernel_3808\2338586223.py:3: DtypeWarning: Columns (32,67,70,71,74,75,78,79,82,83,86,87,90,91,94) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = npi_columns, chunksize = 10000)): #Reading data in by chuncks
C:\Users\tjsim\AppData\Local\Temp\ipykernel_3808\2338586223.py:3: DtypeWarning: Columns (32,71,74,75,78,79,82,83,86,87,90,91,94) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = npi_columns, chunksize = 10000

C:\Users\tjsim\AppData\Local\Temp\ipykernel_3808\2338586223.py:3: DtypeWarning: Columns (32,75,78,79,82,83,86) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = npi_columns, chunksize = 10000)): #Reading data in by chuncks
C:\Users\tjsim\AppData\Local\Temp\ipykernel_3808\2338586223.py:3: DtypeWarning: Columns (75,78,79,82,83,86,87,90,91,94,95,98) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = npi_columns, chunksize = 10000)): #Reading data in by chuncks
C:\Users\tjsim\AppData\Local\Temp\ipykernel_3808\2338586223.py:3: DtypeWarning: Columns (32,75,78,79,82,83,86,87,90,91,94,95,98) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = npi_columns, chunksize = 10000)): #Reading data 

C:\Users\tjsim\AppData\Local\Temp\ipykernel_3808\2338586223.py:3: DtypeWarning: Columns (32,103,106) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = npi_columns, chunksize = 10000)): #Reading data in by chuncks
C:\Users\tjsim\AppData\Local\Temp\ipykernel_3808\2338586223.py:3: DtypeWarning: Columns (32,87,90,91,94) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = npi_columns, chunksize = 10000)): #Reading data in by chuncks
C:\Users\tjsim\AppData\Local\Temp\ipykernel_3808\2338586223.py:3: DtypeWarning: Columns (32,99,102,103,106) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = npi_columns, chunksize = 10000)): #Reading data in by chuncks
C:\Users\tjsim\AppData\Local\Temp\ipyk

C:\Users\tjsim\AppData\Local\Temp\ipykernel_3808\2338586223.py:3: DtypeWarning: Columns (5,6,7,8,10,32,87,90,91,94,95,98,99,102) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = npi_columns, chunksize = 10000)): #Reading data in by chuncks
C:\Users\tjsim\AppData\Local\Temp\ipykernel_3808\2338586223.py:3: DtypeWarning: Columns (5,6,8,10) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = npi_columns, chunksize = 10000)): #Reading data in by chuncks
C:\Users\tjsim\AppData\Local\Temp\ipykernel_3808\2338586223.py:3: DtypeWarning: Columns (5,6,8,10,91,94,95,98,99,102) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = npi_columns, chunksize = 10000)): #Reading data in by chuncks
C:\Use

In [14]:
db.close()

In [11]:
nucc = pd.read_csv('nucc_taxonomy_230.csv', encoding = 'iso-8859-1')

nucc.columns = [x.lower().replace(' ', '_') for x in nucc.columns]

db = sqlite3.connect('hop_teaming.sqlite')

nucc.to_sql('taxonomy', db, if_exists = 'append', index = False)

In [15]:
db.close()

In [17]:
db = sqlite3.connect('hop_teaming.sqlite')

pd.read_excel('ZIP_CBSA_122021.xlsx').to_sql('zipcode', db, if_exists = 'append', index = False)

In [18]:
db.close()

In [ ]:
WITH hospitals_tn AS(
    SELECT 
        npi,
        entity_type_code AS entity,
        `provider_organization_name_(legal_business_name)` AS organization,
        SUBSTR(provider_business_practice_location_address_postal_code, 1, 5) AS zipcode,
        taxonomy_code
    FROM npi_data
    WHERE 
        entity_type_code = 2
)

)
GROUP BY 
    from_tax.grouping,
    from_tax.specialization,
    from_tax.classification,
    to_npi,
    htn.organization,
    to_tax.grouping,
    to_tax.classification,
    to_tax.specialization
ORDER BY total_patient DESC 

In [38]:
query = '''
SELECT
    from_npi,
    from_tax.grouping AS from_grouping,
    from_tax.classification AS from_classification,
    from_tax.specialization AS from_specialization,
    to_npi,
    htn.organization AS to_organization,
    to_tax.grouping AS to_grouping,
    to_tax.classification AS to_classification,
    to_tax.specialization AS to_specialization,
    SUM(patient_count) AS total_patient
FROM npi_data
INNER JOIN hop AS htn
ON to_npi = htn.npi
INNER JOIN npi_data AS fro
ON from_npi = fro.npi
LEFT JOIN taxonomy AS from_tax
ON fro.taxonomy_code = from_tax.code
LEFT JOIN taxonomy AS to_tax
ON htn.taxonomy_code = to_tax.code
WHERE htn.zipcode IN (
    SELECT zip
    FROM zipcode
    WHERE cbsa = 34980
)'''

In [69]:
query = '''
SELECT 
    DISTINCT entity_type_code AS entity,
    npi,
    `provider_organization_name_(legal_business_name)` AS organization,
    SUBSTR(provider_business_practice_location_address_postal_code, 1, 5) AS zipcode,
    taxonomy_code
FROM 
    npi_data
WHERE
    entity_type_code = 2
'''

In [70]:
with sqlite3.connect('hop_teaming.sqlite') as db: 
    profile = pd.read_sql(query, db)



In [71]:
profile

,entity,npi,organization,zipcode,taxonomy_code


In [3]:
db = sqlite3.connect('hop_teaming.sqlite')

db.execute('CREATE INDEX f_npi ON hop(from_npi)')
db.execute('CREATE INDEX t_npi ON hop(to_npi)')

db.close()


OperationalError: index f_npi already exists

In [18]:
db = sqlite3.connect('hop_teaming.sqlite')

db.execute('CREATE INDEX prov_last_name ON npi_data(provider_last_name)')



db.close()

#db.execute('CREATE INDEX prov_first_name ON npi(provider_first_name)')
#db.execute('CREATE INDEX prov_prefix ON npi(provider_name_prefix_text)')
#db.execute('CREATE INDEX prov_suffix ON npi( provider_name_suffix_text)')
#db.execute('CREATE INDEX prov_suffix ON npi( provider_name_suffix_text)')
#db.execute('CREATE INDEX prov_practice_zip ON npi(provider_business_practice_location_address_postal_code)')

#provider_last_name_(legal_name)	provider_first_name	provider_middle_name	
#provider_name_prefix_text provider_name_suffix_text	provider_business_practice_location_address_postal_code

OperationalError: no such column: provider_last_name

In [19]:
query = """
SELECT from_npi, to_npi
FROM hop
LIMIT 1000
"""

In [20]:
with sqlite3.connect('hop_teaming.sqlite') as db: 
    npi_sqlite = pd.read_sql(query, db)

In [21]:
npi_sqlite

,from_npi,to_npi
0,1508062167,1730166109
1,1508065640,1730166109
2,1508052093,1730166109
3,1508172545,1730166109
4,1508285131,1730166109
...,...,...
995,1508868506,1730177452
996,1508864257,1730177452
997,1508875980,1730177452
998,1508870163,1730177478


In [22]:
query = """
SELECT NPI,
entity_type_code,
provider_first_name,
provider_name_prefix_text,
taxonomy_code
FROM npi_data
limit 10000
"""

In [23]:
with sqlite3.connect('hop_teaming.sqlite') as db: 
    npi_sqlite = pd.read_sql(query, db)

In [27]:
npi_sqlite['entity_type_code']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
9995    1.0
9996    1.0
9997    1.0
9998    1.0
9999    1.0
Name: entity_type_code, Length: 10000, dtype: float64

In [ ]:
 'Provider Middle Name',
    provider_name_prefix_text',
    'Provider Name Suffix Text',
    'Provider Credential Text',
    'Provider First Line Business Practice Location Address',
    'Provider Second Line Business Practice Location Address',
    'Provider Business Practice Location Address City Name',
    'Provider Business Practice Location Address State Name',
    'Provider Business Practice Location Address Postal Code',

In [9]:
zipcode_sqlite

,zip,cbsa,usps_zip_pref_city,usps_zip_pref_state,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,683,41900,SAN GERMAN,PR,0.999842,1.0,1.0,0.999855
1,683,32420,SAN GERMAN,PR,0.000158,0.0,0.0,0.000145
2,923,41980,SAN JUAN,PR,1.000000,1.0,1.0,1.000000
3,1010,44140,BRIMFIELD,MA,0.976896,1.0,1.0,0.977816
4,1010,49340,BRIMFIELD,MA,0.023104,0.0,0.0,0.022184
...,...,...,...,...,...,...,...,...
47479,60684,16980,CHICAGO,IL,0.000000,1.0,0.0,1.000000
47480,33945,15980,PINELAND,FL,0.000000,0.0,1.0,1.000000
47481,78144,99999,PANNA MARIA,TX,0.000000,1.0,0.0,1.000000
47482,12257,10580,ALBANY,NY,0.000000,1.0,0.0,1.000000


In [7]:
taxonomy_sqlite

,code,grouping,classification,specialization,definition,notes,display_name,section
0,193200000X,Group,Multi-Specialty,None,A business group of one or more individual pra...,[7/1/2003: new],Multi-Specialty Group,Individual
1,193400000X,Group,Single Specialty,None,A business group of one or more individual pra...,[7/1/2003: new],Single Specialty Group,Individual
2,207K00000X,Allopathic & Osteopathic Physicians,Allergy & Immunology,None,An allergist-immunologist is trained in evalua...,"Source: American Board of Medical Specialties,...",Allergy & Immunology Physician,Individual
3,207KA0200X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Allergy,"A physician who specializes in the diagnosis, ...",Source: National Uniform Claim Committee,Allergy Physician,Individual
4,207KI0005X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Clinical & Laboratory Immunology,An allergy and immunology physician who specia...,"Source: National Uniform Claim Committee, 2022...",Clinical & Laboratory Immunology (Allergy & Im...,Individual
...,...,...,...,...,...,...,...,...
868,343800000X,Transportation Services,Secured Medical Transport (VAN),None,A public or privately owned transportation ser...,None,Secured Medical Transport (VAN),Non-Individual
869,344600000X,Transportation Services,Taxi,None,A land commercial vehicle used for the transpo...,None,Taxi,Non-Individual
870,347D00000X,Transportation Services,Train,None,An organization or business licensed to provid...,None,Train,Non-Individual
871,347E00000X,Transportation Services,Transportation Broker,None,An organization that provides transportation f...,Source: Section 6083 of the Deficit Reduction ...,Transportation Broker,Non-Individual


In [10]:
npi_sqlite

,from_npi,to_npi
0,1508062167,1730166109
1,1508065640,1730166109
2,1508052093,1730166109
3,1508172545,1730166109
4,1508285131,1730166109
...,...,...
217911303,1417903683,1497940662
217911304,1417909367,1497940662
217911305,1417909029,1497940662
217911306,1417916610,1497940662


In [4]:
SELECT 
npi
FROM hop
sqlite_schema
WHERE 
type ='table' AND name NOT LIKE 'sqlite_%';

SyntaxError: invalid syntax (2624536305.py, line 3)